# OCR using CNN

Importing the libraries

In [64]:
from keras.layers import Dense, Dropout, Convolution2D, MaxPooling2D
from keras.layers import Flatten
from keras.models import Sequential
import scipy
import numpy as np
import os
import keras.utils

Loading data form EMNIST database

In [65]:
EMNIST = scipy.io.loadmat(os.path.join(os.getcwd(), "matlab", "emnist-digits.mat"))

x_train = EMNIST["dataset"][0][0][0][0][0][0].astype("float64")
y_train = EMNIST["dataset"][0][0][0][0][0][1]

x_test = EMNIST['dataset'][0][0][1][0][0][0].astype("float64")
y_test = EMNIST['dataset'][0][0][1][0][0][1]

# Scaling data
x_train = (x_train - np.mean(x_train)) /np.std(x_train)
x_test = (x_test - np.mean(x_train)) /np.std(x_train)

nb_classes = 10

y_train = keras.utils.to_categorical(y_train, nb_classes)
y_test = keras.utils.to_categorical(y_test, nb_classes)

print("Fraction test {}, Fraction train {}".format(len(y_test)/(len(y_test) + len(y_train)), len(y_train)/(len(y_test) + len(y_train))))

0.14285714285714285 0.8571428571428571


In [66]:
height = 28
width = 28
input_shape = (height, width, 1) # Dimensions of the input image : 28 x 28, third dimension 1 is added for compatibility with Convolution2D function

x_train_scaled = x_train[:len(x_train)].reshape(len(x_train), height, width, 1)
x_test_scaled = x_test[:len(x_test)].reshape(len(x_test), height, width, 1)

In [67]:
activation = "relu"


nb_filters = 32 # number of convolutional filters to use
pool_size = (2, 2) # size of pooling area for max pooling
kernel_size = (3, 3) # convolution kernel size
model = Sequential()
model.add(Convolution2D(nb_filters,
                        kernel_size,
                        padding='valid',
                        input_shape=input_shape,
                        activation='relu'))
model.add(Convolution2D(nb_filters,
                        kernel_size,
                        activation='relu'))

model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
            optimizer='adadelta',
            metrics=['accuracy'])

batch = 256
epoch = 10
model.fit(x_train_scaled, y_train,
            batch_size=batch,
            epochs=epoch,
            verbose=1,
            validation_data=(x_test_scaled, y_test))

score = model.evaluate(x_test_scaled, y_test, verbose=0)


c:\Users\msii\lepl1109\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 246s 257ms/step - accuracy: 0.1499 - loss: 2.2734 - val_accuracy: 0.5618 - val_loss: 2.0899
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 308s 328ms/step - accuracy: 0.4163 - loss: 2.0392 - val_accuracy: 0.7633 - val_loss: 1.6331
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 309s 329ms/step - accuracy: 0.6350 - loss: 1.5604 - val_accuracy: 0.8224 - val_loss: 0.9751
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 244s 246ms/step - accuracy: 0.7303 - loss: 1.0307 - val_accuracy: 0.8497 - val_loss: 0.6294
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 280s 265ms/step - accuracy: 0.7781 - loss: 0.7641 - val_accuracy: 0.8681 - val_loss: 0.4901
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 298s 318ms/step - accuracy: 0.8069 - loss: 0.6425 - val_accuracy: 0.8799 - val_loss: 0.4196
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 209s 222ms/step - accuracy: 0.8274 - loss: 0.5650 - val_accuracy: 0.8911 - val_loss: 0.3768
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 259s 276ms/step - accuracy: 0.8418 -

In [71]:
y_pred = model.predict(x_train_scaled[1].reshape(1, height, width, 1))

print(y_train[1])
print(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[[1.6591708e-05 7.1169794e-05 2.1802531e-05 1.8994453e-04 1.3521720e-03
  5.8275950e-04 5.4246851e-07 3.6679428e-02 3.0001642e-02 9.3108404e-01]]
